In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import threading

In [2]:
irisDataset = pd.read_csv('irisDataset.csv')

In [3]:
irisDataset.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [4]:
X = irisDataset.drop(['species'], axis=1)
y = irisDataset['species']

In [5]:
X = X.to_numpy()
y = y.to_numpy()


In [6]:
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(y)


In [7]:
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)

In [8]:
class Node:
    def __init__(self, threshold=None, value=None, leftChild=None, rightChild=None, feature=None):
        self.leftChild = leftChild
        self.rightChild = rightChild
        self.threshold = threshold
        self.value = value
        self.feature = feature

class decisionTreeClassifier:
    def __init__(self, minSamplesSplit, minSamplesLeaf, maxDepth, X, y, features=None):
        '''
        minSamplesSplit -> amt samples needed to consider node for being split
        minSamplesLeaf -> amt samples needed in a leaf after a split
        '''
        self.minSamplesSplit = minSamplesSplit # amt samples needed to consider node for being split
        self.minSamplesLeaf = minSamplesLeaf # amt samples needed in a leaf after a split
        self.maxDepth = maxDepth
        self.X = X
        self.y = y

        if features is None:
            self.features = range(self.X.shape[1])
        else:
            self.features = features

        self.root = Node()

    def printTree(self):
        def recursePrint(node, depth):
            indent = "  " * depth
            if node.value is not None:
                print(f"{indent}Leaf -> Predict {node.value}")
                return
            print(f"{indent}if x[{node.feature}] <= {node.threshold}:")
            recursePrint(node.leftChild, depth + 1)
            print(f"{indent}else:")
            recursePrint(node.rightChild, depth + 1)

        recursePrint(self.root, 0)

    def createTree(self, curNode:Node, depth, subset):
        if depth > self.maxDepth or len(subset) <= self.minSamplesSplit:
            return Node(value=self.mostOccurrentSample(subset))
        parentEntropy = self.entropy(subset)

        logs = []
        for feature in self.features:
            for threshold in range(subset.shape[0]):
                threshold = subset[threshold, feature]
                leftSubset, rightSubset = self.split(subset, feature, threshold)

                if len(leftSubset) >= self.minSamplesLeaf and len(rightSubset) >= self.minSamplesLeaf:
                    weightedEntropy = (len(leftSubset)/len(subset))*(self.entropy(leftSubset)) \
                    + (len(rightSubset)/len(subset))*(self.entropy(rightSubset))
                
                    
                    informationGain = parentEntropy - weightedEntropy
                    logs.append((feature, threshold, informationGain))
        
        if not logs:
            # becomes a leaf node
            return Node(value=self.mostOccurrentSample(subset))

        logs.sort(key=lambda x: x[-1], reverse=True)
        best = logs[0]

        leftSubset, rightSubset = self.split(subset, best[0], best[1])

        curNode.threshold = best[1]
        curNode.feature = best[0]

        curNode.leftChild, curNode.rightChild = self.createTree(Node(), depth+1, leftSubset), self.createTree(Node(), depth+1, rightSubset)
        return curNode

    def mostOccurrentSample(self, subset):
        values, counts = np.unique(subset[:, -1], return_counts=True)
        return values[np.argmax(counts)]

    def split(self, subset, feature, threshold):
        left = np.array(np.empty((0, subset.shape[1])))
        right = np.array(np.empty((0, subset.shape[1])))
        for row in range(subset.shape[0]):
            item = subset[row, feature]
            
            if item <= threshold:
                left = np.vstack((left, np.expand_dims(subset[row], 0)))
            else:
                right = np.vstack((right, np.expand_dims(subset[row], 0)))

        return [left, right]
    
    def entropy(self, subset):
        # look at the y values
        values, counts = np.unique(subset[:, -1], return_counts=True)
        probabilities = counts / counts.sum()
        return -np.sum(probabilities * np.log2(probabilities))
    
    def predict(self, x):
        if x.shape == (self.X.shape[1],):
            # only one data point to predict
            x = np.expand_dims(x, 0)

        predictions = np.array([])
        for xInput in x:
            curNode = self.root
            while curNode.value is None:
                # print(curNode.threshold, curNode.feature, curNode.value)
                
                if xInput[curNode.feature] >= curNode.threshold:
                    curNode = curNode.rightChild
                else:
                    curNode = curNode.leftChild
            predictions = np.append(predictions, curNode.value)
        return predictions
    
    def fit(self):
        subset = np.concatenate((self.X, self.y.reshape(-1, 1)), axis=1)

        self.root = self.createTree(self.root, 0, subset)


class RandomForestClassifier:
    def __init__(self, X, y, minSamplesSplit, minSamplesLeaf, maxDepth, nEstimators, bootstrap=True, nJobs=-1):
        self.minSamplesSplit = minSamplesSplit
        self.minSamplesLeaf = minSamplesLeaf 
        self.nEstimators = nEstimators
        self.maxDepth = maxDepth
        self.bootstrap = bootstrap
        self.X, self.y = X, y

        self.binFeatureAmount = int(np.ceil(self.X.shape[1] ** 0.5))
    
    def bootstrapData(self):
        # randomly pick data with replacing to different estimators using different features
        data = np.concatenate((self.X, self.y.reshape(-1, 1)), axis=1)       
        bins = []
        for _ in range(self.nEstimators):
            # use the choice to get the data, combine with the amt of features for the finished bin
            binDataIndices = np.random.choice(list(range(0, data.shape[0])), data.shape[0], replace=True)
            binData = data[binDataIndices]
            
            featuresIndices = list(range(0, data.shape[1]-1))
            features = np.random.choice(featuresIndices, self.binFeatureAmount, replace=False)
            features = features.reshape(1, -1)
            features = np.tile(features, (data.shape[0], 1))[:data.shape[0]]
        
            binData = np.hstack((binData, features))
            
            bins.append(binData)     
        return bins       

    def fit(self):        
        bins = self.bootstrapData()
        estimators = []
        for estimatorIdx in tqdm(range(self.nEstimators)):
            curBin = bins[estimatorIdx]
            data, features = curBin[:, :(curBin.shape[1] - self.binFeatureAmount)], curBin[:, (curBin.shape[1] - self.binFeatureAmount):]
            features = features.astype(int)
            features = features[0, :]

            curX, curY = data[:, :self.X.shape[1]], data[:, self.X.shape[1]:]
            curY = curY.astype(int)
            curY = curY.squeeze()

            estimator = decisionTreeClassifier(
                minSamplesSplit=self.minSamplesSplit,
                minSamplesLeaf=self.minSamplesLeaf,
                maxDepth=self.maxDepth,
                X=curX,
                y=curY,
                features=features
            )
            
            estimator.fit()
            estimators.append(estimator)
            
        self.estimators = estimators

            
    def predict(self, x):
        if x.shape == (self.X.shape[1],):
            # only one data point to predict
            x = np.expand_dims(x, 0)

        predictions = np.array([])
        for xInput in x:
            estimatorPredictionsToBeAnalyzed = []
            for estimator in self.estimators:
                estimatorPredictionsToBeAnalyzed.append(estimator.predict(xInput))

            values, counts = np.unique(estimatorPredictionsToBeAnalyzed, return_counts=True)
            estimatorPredictionsMajorityVote = values[np.argmax(counts)]
            
            predictions = np.append(predictions, estimatorPredictionsMajorityVote)
        return predictions

In [9]:
dtc = decisionTreeClassifier(minSamplesSplit=25, minSamplesLeaf=3, maxDepth=3, X=xTrain, y=yTrain)
dtc.fit()

In [10]:
rfc = RandomForestClassifier(
    X=xTrain, y=yTrain, 
    minSamplesSplit=dtc.minSamplesSplit, minSamplesLeaf=dtc.minSamplesLeaf, 
    maxDepth=dtc.maxDepth, nEstimators=100, 
    bootstrap=True,
    nJobs=-1
)

rfc.fit()

100%|██████████| 100/100 [00:26<00:00,  3.79it/s]


In [11]:
dtc.printTree()

if x[2] <= 1.9:
  if x[0] <= 5.0:
    Leaf -> Predict 0.0
  else:
    Leaf -> Predict 0.0
else:
  if x[2] <= 4.7:
    if x[3] <= 1.5:
      if x[0] <= 5.8:
        Leaf -> Predict 1.0
      else:
        Leaf -> Predict 1.0
    else:
      Leaf -> Predict 1.0
  else:
    if x[2] <= 5.0:
      Leaf -> Predict 2.0
    else:
      if x[0] <= 7.6:
        Leaf -> Predict 2.0
      else:
        Leaf -> Predict 2.0


In [12]:
yPred = dtc.predict(xTrain).astype(int)

trainDTCError = np.sum(yTrain != yPred) / len(yPred)
trainDTCError

0.09166666666666666

In [13]:
yPred = dtc.predict(xTest).astype(int)
testDTCError = np.sum(yTest != yPred) / len(yPred)
testDTCError

0.1

In [14]:
trainDTCPreds = dtc.predict(xTrain).astype(int)
testDTCPreds = dtc.predict(xTest).astype(int)

print("Train acc:", np.mean(trainDTCPreds == yTrain))
print("Test acc:", np.mean(testDTCPreds == yTest))

Train acc: 0.9083333333333333
Test acc: 0.9


In [15]:
yPred = rfc.predict(xTrain).astype(int)

trainRfcError = np.sum(yTrain != yPred) / len(yPred)
trainRfcError

0.05

In [16]:
yPred = rfc.predict(xTest).astype(int)
testRfcError = np.sum(yTest != yPred) / len(yPred)
testRfcError

0.06666666666666667

In [17]:
trainRfcPreds = rfc.predict(xTrain).astype(int)
testRfcPreds = rfc.predict(xTest).astype(int)

print("Train acc:", np.mean(trainRfcPreds == yTrain))
print("Test acc:", np.mean(testRfcPreds == yTest))

Train acc: 0.95
Test acc: 0.9333333333333333


In [18]:
print(f'''
                    DTC           RFC
Train Error    ->   {format(trainDTCError, ".2f")}          {format(trainRfcError, ".2f")}
Test  Error    ->   {format(testDTCError, ".2f")}          {format(testRfcError, ".2f")}

Train Accuracy ->   {format(np.mean(trainDTCPreds == yTrain), ".2f")}          {format(np.mean(trainRfcPreds == yTrain), ".2f")}
Test  Accuracy ->   {format(np.mean(testDTCPreds == yTest), ".2f")}          {format(np.mean(testRfcPreds == yTest), ".2f")}
''')


                    DTC           RFC
Train Error    ->   0.09          0.05
Test  Error    ->   0.10          0.07

Train Accuracy ->   0.91          0.95
Test  Accuracy ->   0.90          0.93

